# Simpleng Tagapag-uri ng Imahe

Ipinapakita ng notebook na ito kung paano mag-uri ng mga imahe gamit ang isang pre-trained neural network.

**Ano ang matututunan mo:**
- Paano mag-load at gumamit ng pre-trained na modelo
- Pagproseso ng imahe
- Paggawa ng prediksyon sa mga imahe
- Pag-unawa sa mga confidence score

**Gamit:** Tukuyin ang mga bagay sa mga imahe (tulad ng "pusa", "aso", "kotse", atbp.)

---


## Hakbang 1: I-import ang Mga Kinakailangang Library

I-import natin ang mga tool na kailangan natin. Huwag mag-alala kung hindi mo pa naiintindihan ang lahat ng ito!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Hakbang 2: I-load ang Pre-trained Model

Gagamitin natin ang **MobileNetV2**, isang neural network na na-train na sa milyon-milyong mga imahe.

Ito ay tinatawag na **Transfer Learning** - paggamit ng model na na-train na ng iba!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Hakbang 3: Mga Helper Function

Gumawa tayo ng mga function para mag-load at maghanda ng mga imahe para sa ating modelo.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Hakbang 4: Subukan sa Mga Sample na Larawan

Subukan nating i-classify ang ilang mga larawan mula sa internet!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Iklasipika ang Bawat Imahe


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Hakbang 5: Subukan ang Sarili Mong Mga Imahe!

Palitan ang URL sa ibaba ng anumang URL ng imahe na nais mong i-klasipika.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Ano ang Nangyari?

1. **Nag-load tayo ng pre-trained na modelo** - Ang MobileNetV2 ay sinanay gamit ang milyon-milyong larawan
2. **Pinroseso natin ang mga larawan** - Iniresize at inayos para sa modelo
3. **Gumawa ng prediksyon ang modelo** - Nagbigay ito ng mga probabilidad para sa 1000 klase ng mga bagay
4. **Ikinonvert natin ang resulta** - Ginawang mas madaling maunawaan ang mga numero sa pamamagitan ng mga label

### Pag-unawa sa Confidence Scores

- **90-100%**: Napakataas na kumpiyansa (halos tiyak na tama)
- **70-90%**: Mataas na kumpiyansa (malamang tama)
- **50-70%**: Katamtamang kumpiyansa (maaaring tama)
- **Mas mababa sa 50%**: Mababa ang kumpiyansa (hindi tiyak)

### Bakit maaaring mali ang mga prediksyon?

- **Hindi karaniwang anggulo o ilaw** - Ang modelo ay sinanay gamit ang mga tipikal na larawan
- **Maraming bagay sa larawan** - Inaasahan ng modelo na may isang pangunahing bagay
- **Bihirang bagay** - Ang modelo ay may kaalaman lamang sa 1000 kategorya
- **Mababang kalidad ng larawan** - Mas mahirap basahin ang malabo o pixelated na mga larawan

---


## 🚀 Mga Susunod na Hakbang

1. **Subukan ang iba't ibang larawan:**
   - Maghanap ng mga larawan sa [Unsplash](https://unsplash.com)
   - I-right-click → "Copy image address" para makuha ang URL

2. **Mag-eksperimento:**
   - Ano ang mangyayari sa abstract art?
   - Kaya bang kilalanin ang mga bagay mula sa iba't ibang anggulo?
   - Paano nito hinahandle ang maraming bagay sa isang larawan?

3. **Matuto pa:**
   - Tuklasin ang [mga aralin sa Computer Vision](../lessons/4-ComputerVision/README.md)
   - Matutong mag-train ng sarili mong image classifier
   - Unawain kung paano gumagana ang CNNs (Convolutional Neural Networks)

---

## 🎉 Binabati Kita!

Kakagawa mo lang ng isang image classifier gamit ang makabagong neural network!

Ang parehong teknik na ito ang ginagamit sa:
- Google Photos (pag-aayos ng iyong mga larawan)
- Mga self-driving na kotse (pagkilala sa mga bagay)
- Medikal na pagsusuri (pag-analyze ng X-rays)
- Quality control (pag-detect ng mga depekto)

Patuloy na mag-eksplora at matuto! 🚀



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
